## Prepping data for learning
### Method: every hour of the day labeled for sunrise and sunset, big batch approach (model 1)

inputs: 
- csv of dates separated on new line
- csv of weather data to label

output: 
- labeled weather data

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

### Sunrise Date Dictionary
Read in the dates csv to make a dictionary of all dates with sunrise photos

dictionary structure: 
- key is date
- value is a tuple of booleans (sunrise = 0, sunset = 0)
- 0 for no photos exist, 1 for photo exists

In [18]:
# sunrise_path = os.path.join(os.getcwd(),"data2011-2017", "denver_sunrise.csv")
# sunset_path = os.path.join(os.getcwd(),"data2011-2017", "denver_sunset.csv")
# weather_path = os.path.join(os.getcwd(),"data2011-2017", "weather_data.csv")
# output_path = 'prepped_data_1.csv'

sunrise_path = os.path.join(os.getcwd(),"data2019", "denver_sunrise.csv")
sunset_path = os.path.join(os.getcwd(),"data2019", "denver_sunset.csv")
weather_path = os.path.join(os.getcwd(),"data2019", "weather_data.csv")
output_path = 'test_data_1.csv'


In [19]:
# get a dictionary of the sunrise and sunset times
# this will be used to verify whether the photo datetime is accurate or not.
# any date whose time is not within an hour of sunrise or sunset will be taken out of the dataset

times = pd.read_csv(weather_path, usecols = ['Sunrise', 'Sunset'] )
sunrise_times = {}
for datetime in times['Sunrise']: 
    date = datetime[:10]
    hour = int(datetime[11:13])
    minute = int(datetime[14:16])
    time = hour * 60 + minute
    sunrise_times[date] = time 
    
sunset_times = {}
for datetime in times['Sunset']: 
    date = datetime[:10]
    hour = int(datetime[11:13])
    minute = int(datetime[14:16])
    time = hour * 60 + minute
    sunset_times[date] = time 
    

In [20]:
# create a dictionary of all dates with photos
dates = {}

# sunrise
df = pd.read_csv(sunrise_path)
  
for column in df: 
    columnSeriesObj = df[column]
    for item in columnSeriesObj.values:
        date = item[:10]
        hour = int(item[11:13])
        minute = int(item[14:16])
        photo_time = hour * 60 + minute
    
        real_time = sunrise_times.get(date)
        # check alleged time's proximity to the real sunrise time to make sure that this is a valid date
        # if real time -60 < photo time < real time + 60
        if real_time - 60 < photo_time < real_time + 60:  
            dates[date] = (1,0)

# sunset
df = pd.read_csv(sunset_path)

# if it is already in dictionary,
# keep sunrise value in the tuple, make the sunset value a 1
# if not in the dictionary,
# add to the dictionary with 0 for sunrise value
for column in df: 
    columnSeriesObj = df[column]
    for item in columnSeriesObj.values:
        date = item[:10]
        hour = int(item[11:13])
        minute = int(item[14:16])
        photo_time = hour * 60 + minute
        real_time = sunset_times.get(date)
        # if valid:
        if real_time - 60 < photo_time < real_time + 60: 
            if date in dates: 
                dates[date] = (dates[date][0], 1) 

            else:
                dates[date] = (0,1)
        
        

In [21]:
# # create a dictionary of all dates with photos
# dates = {}

# # sunrise
# df = pd.read_csv(sunrise_path)

# for column in df: 
#     columnSeriesObj = df[column]
#     for item in columnSeriesObj.values:
#         date = item[:10]
#         # if it is within sunrise times 
#         dates[date] = (1,0)
    
# # sunset
# df = pd.read_csv(sunset_path)

# for column in df: 
#     columnSeriesObj = df[column]
#     for item in columnSeriesObj.values:
#         date = item[:10]
#         # if it is already in dictionary,
#         # keep sunrise value in the tuple, make the sunset value a 1
#         if date in dates: 
#             dates[date] = (dates[date][0], 1) 
#         # if not in the dictionary,
#         # add to the dictionary with 0 for sunrise value
#         else:
#             dates[date] = (0,1)
# og_dates = dates


denver cleaning: 

- 396 sunrise dates removed
- 570 sunset dates removed
- 816 remained the same


### Clean the weather data
- read in the csv with full weather data as a pandas dataframe


In [22]:
input_file = pd.read_csv(weather_path)
# input_file

find out what conditions exist

In [23]:
output = pd.read_csv(weather_path)
conditions = {}
for row in output['Conditions']:
    for item in row.split(', '):
        if item not in conditions: 
            conditions[item] = 0
        else: 
            conditions[item] += 1
            

types = {}
for row in output['Weather Type']:
    if isinstance(row, str):
        for item in row.split(', '):
            if item not in types: 
                types[item] = 1
            else: 
                types[item] += 1

# make a new column in this order for every condition
for condition in list(conditions):
    output.at[:,"Condition: " + condition] = 0.0
    
# make a new column in this order for every weather type
for type in list(types):
    output.at[:,"Type: " + type] = 0.0

In [24]:
output = pd.read_csv(weather_path)

# rename sunrise and sunset date time columns 
output.rename(columns = {'Sunrise':'Sunrise Time', 'Sunset':'Sunset Time'}, inplace = True)


# # change to sunrise times
# # https://note.nkmk.me/en/python-pandas-str-slice/
# hour = output['Sunrise Time'].str[11:13]
# minute = output['Sunrise Time'].str[14:16]
# maybe convert it into a floating point which represents current time in minutes / total minutes in a day


# change conditions into one conditions vector with 0s and 1s --> Condition: <condition>
for i, row in enumerate(output['Conditions']):
    for condition in row.split(', '):
        output.at[i,"Condition: " + condition] = 1

# # weather types --> Type: <weather_type>
# for i, row in enumerate(output['Weather Type']):
#         if isinstance(row, str):
#             for type in row.split(', '):
#                 output.at[i,"Type: " +type] = 1

# add sunrise and sunset labels based on the dates dictionary
for i, row in enumerate(output['Date time']):
    date = row[:10]
    
    # get date data
    output.at[i, "Month"] = int(row[5:7])
    output.at[i, "Hour"] = int(row[-8:-6])
    
    
    # check the dates dictionary 
    # if the first value in tuple is 1, label yes sunrise, if second, label yes sunset
    if date in dates: 
        if dates.get(date)[0]:
            output.at[i,'Sunrise']= 1.0
        if dates.get(date)[1]:
            output.at[i,'Sunset']= 1.0
        
# populate NaNs with 0.0s
output = output.fillna(0)

# delete any column with strings or unecessary info
output = output.drop(['Moon Phase', 'Minimum Temperature', 'Precipitation Cover', 'Maximum Temperature', 'Latitude', 'Longitude', 'Date time', 'Info', 'Address', 'Conditions', 'Weather Type', 'Sunrise Time', 'Sunset Time', 'Resolved Address', 'Name'], axis=1)


In [25]:
# # iterating the columns
# list(output)

In [26]:
# index = False makes it ignore the first column
output.to_csv(output_path, index=False) 
